In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existin

# YOLOv11

# YOLOv11 (DS v1 acne-v8)

In [7]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="d0TYZxZJCWWTuBFwxYNI")
project = rf.workspace("asllanguage").project("acne-yolo-v8-zuqxc")
version = project.version(1)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Acne-Yolo-v8-1 in yolov11:: 100%|██████████| 5216/5216 [00:00<00:00, 8649.22it/s]


In [ ]:
from ultralytics import YOLO
import ultralytics.data as data
import ultralytics.data.dataset as dataset
import ultralytics.data.build as build

import numpy as np

class YOLOWeightedDataset(data.dataset.YOLODataset):
    def __init__(self, *args, data=None, task="train", **kwargs):
        """
        Initialize the WeightedDataset.

        Args:
            class_weights (list or numpy array): A list or array of weights corresponding to each class.
        """
    
        super(YOLOWeightedDataset, self).__init__(*args, data=data, task=task, **kwargs)

        self.train_mode = "train" in self.prefix

        # You can also specify weights manually instead
        self.count_instances()
        class_weights = np.sum(self.counts) / self.counts

        # Aggregation function
        self.agg_func = np.mean

        self.class_weights = np.array(class_weights)
        self.weights = self.calculate_weights()
        self.probabilities = self.calculate_probabilities()

    ... # rest of the code from https://y-t-g.github.io/tutorials/yolo-class-balancing/


dataset.YOLODataset = YOLOWeightedDataset

model = YOLO("yolo11n.pt")

train_results1 = model.train(
    data='/kaggle/working/Acne-Yolo-v8-1/data.yaml',
    epochs=200,              # Meningkatkan jumlah epoch
    device=0,
    imgsz=640,
    save_period=20,
    lr0=0.001,               # Tingkatkan learning rate awal
    lrf=0.01,                # Ujung learning rate (lebih besar agar tidak terlalu mendatar)
    cos_lr=True,             # Tetap gunakan Cosine LR
    warmup_epochs=3,         # Tambahkan warmup untuk stabilisasi awal
    patience=50,             # Biarkan model mencoba lebih lama sebelum early stop
    optimizer="AdamW",       # Optimizer yang lebih stabil dari SGD untuk dataset kecil
    seed=42,
)


Ultralytics 8.3.153 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/Acne-Yolo-v8-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=190, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0,

train: Scanning /kaggle/working/Acne-Yolo-v8-1/train/labels.cache... 2400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2400/2400 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 54, len(boxes) = 31091. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 269.4±46.9 MB/s, size: 40.6 KB)


val: Scanning /kaggle/working/Acne-Yolo-v8-1/valid/labels.cache... 101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 101/101 [00:00<?, ?it/s]

...

    190/190      7.28G     0.9799     0.5253     0.9384        333        640: 100%|██████████| 75/75 [00:26<00:00,  2.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.78it/s]

                   all        101       1204      0.706      0.597      0.614      0.309



190 epochs completed in 1.501 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 5.5MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.153 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.82it/s]
invalid value encountered in less
invalid value encountered in less


                   all        101       1204      0.715      0.591      0.623      0.316
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train5


In [13]:
!zip -r "/kaggle/working/model(12_06_01).zip" /kaggle/working/runs/detect/train5

  adding: kaggle/working/runs/detect/train5/ (stored 0%)
  adding: kaggle/working/runs/detect/train5/train_batch13500.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/train5/args.yaml (deflated 51%)
  adding: kaggle/working/runs/detect/train5/train_batch13501.jpg (deflated 8%)
  adding: kaggle/working/runs/detect/train5/train_batch0.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/train5/train_batch1.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/train5/labels_correlogram.jpg (deflated 32%)
  adding: kaggle/working/runs/detect/train5/P_curve.png (deflated 17%)
  adding: kaggle/working/runs/detect/train5/val_batch0_pred.jpg (deflated 7%)
  adding: kaggle/working/runs/detect/train5/weights/ (stored 0%)
  adding: kaggle/working/runs/detect/train5/weights/epoch150.pt (deflated 28%)
  adding: kaggle/working/runs/detect/train5/weights/epoch0.pt (deflated 29%)
  adding: kaggle/working/runs/detect/train5/weights/epoch180.pt (deflated 28%)
  adding: kaggle/working/runs/detec